In [3]:
!pip install catboost #코랩환경시 설치 필요

import pandas as pd
import numpy as np
import datetime
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
import random
import os
from catboost import CatBoostClassifier
from imblearn.over_sampling import ADASYN
import pickle
import joblib


train_path = '/content/drive/MyDrive/DKU_DATA/train_data.csv' #학습셋 경로
test_path = '/content/drive/MyDrive/DKU_DATA/test_data.csv' #테스트셋 경로

bin_model_path = '/content/bin_classifier.pkl' #이진분류 모델 경로
mul_model_path = '/content/multi_classifier.pkl' #다중분류 모델 경로


###############################################################################
###################################이진 분류 시작##################################
###############################################################################



#############################학습 데이터 호출(이진 분류)#############################

# CSV 파일을 DataFrame으로 읽어오기 (인코딩 방식 지정)
bin_train_df = pd.read_csv(train_path, encoding='cp949')

#0이 아닌 컬럼은 모두 1로 변환, 이진분류를 위해서.
bin_train_df.loc[bin_train_df['재실인원'] != 0, '재실인원'] = 1

# null 값을 포함하는 행(row) 제거
bin_train_df = bin_train_df.dropna()

# 중복된 행(row) 제거
bin_train_df = bin_train_df.drop_duplicates()

# regdate 컬럼을 datetime 형식으로 변환 후 시간을 반올림하여 0~23 사이의 값으로 변환
bin_train_df['regdate'] = pd.to_datetime(bin_train_df['regdate'], format='%Y-%m-%d %H:%M:%S')
bin_train_df['regdate'] = bin_train_df['regdate'].apply(lambda x: (x + datetime.timedelta(minutes=30)).hour % 24)

# ADASYN 오버 샘플링 적용
adasyn = ADASYN(random_state=28,sampling_strategy='auto')
X_oversampled, y_oversampled = adasyn.fit_resample(bin_train_df.drop('재실인원', axis=1), bin_train_df['재실인원'])

# 새로운 오버샘플링된 데이터프레임 생성
df_oversampled = pd.DataFrame(X_oversampled, columns=bin_train_df.drop('재실인원', axis=1).columns)
df_oversampled['재실인원'] = y_oversampled

bin_train_df = df_oversampled

# 최종 선택된 컬럼
selected_columns = ['temp', 
                    'humi', 
                    'co2', 
                    'eco2',
                    '재실인원']

# 최종 학습 데이터
bin_train_data = bin_train_df[selected_columns]

##########################################################################



##########################테스트 데이터 호출(이진 분류)##########################

# CSV 파일을 DataFrame으로 읽어오기 (인코딩 방식 지정)
bin_test_df = pd.read_csv(test_path, encoding='cp949')

# 0이 아닌 컬럼은 모두 1로 변환, 이진분류를 위해서.
bin_test_df.loc[bin_test_df['재실인원'] != 0, '재실인원'] = 1

# regdate 컬럼을 datetime 형식으로 변환 후 시간을 반올림하여 0~23 사이의 값으로 변환
bin_test_df['regdate'] = pd.to_datetime(bin_test_df['regdate'], format='%Y-%m-%d %H:%M:%S')
bin_test_df['regdate'] = bin_test_df['regdate'].apply(lambda x: (x + datetime.timedelta(minutes=30)).hour % 24)

# bin_train_data 데이터와 동일한 컬럼 사용, 최종 학습데이터
bin_test_df = bin_test_df[bin_train_data.columns]

# train_test_data에서 각 컬럼의 최빈값 계산
mode_values = bin_train_data.mode().iloc[0]

# test_test_data의 결측치를 최빈값으로 채우기
bin_test_data = bin_test_df.fillna(mode_values)

##########################################################################



###########################이진 분류 모델 호출 및 평가###########################

# 학습 데이터셋
bin_y_train = bin_train_data['재실인원']
bin_X_train = bin_train_data.drop('재실인원', axis=1)

# 테스트 데이터셋 
bin_y_test = bin_test_data['재실인원']
bin_X_test = bin_test_data.drop('재실인원', axis=1)

# 저장된 모델 피클 파일을 불러오기
bin_model = joblib.load(bin_model_path)

# 예측 수행
y_pred_prob = bin_model.predict_proba(bin_X_test)[:, 1]  # 클래스 1의 예측 확률을 가져옴
y_pred = np.where(y_pred_prob >= 0.5, 1, 0)  # 예측 확률을 기준으로 이진 예측으로 변환

print()
# 정확도 계산
accuracy = accuracy_score(bin_y_test, y_pred)
print("이진분류 Accuracy:", accuracy)

# ROC AUC 계산
roc_auc = roc_auc_score(bin_y_test, y_pred_prob)
print("이진분류 ROC AUC:", roc_auc)
print()

##########################################################################



###############################################################################
####################################이진 분류 끝##################################
###############################################################################








##############################################################################
#################################다중 분류 시작##################################
##############################################################################



#############################학습 데이터 호출(다중 분류)#############################

# CSV 파일을 DataFrame으로 읽어오기 (인코딩 방식 지정)
mul_train_df = pd.read_csv(train_path, encoding='cp949')

# null 값을 포함하는 행(row) 제거
mul_train_df = mul_train_df.dropna()

# 중복된 행(row) 제거
mul_train_df = mul_train_df.drop_duplicates()

# regdate 컬럼을 datetime 형식으로 변환 후 시간을 반올림하여 0~23 사이의 값으로 변환
mul_train_df['regdate'] = pd.to_datetime(mul_train_df['regdate'], format='%Y-%m-%d %H:%M:%S')
mul_train_df['regdate'] = mul_train_df['regdate'].apply(lambda x: (x + datetime.timedelta(minutes=30)).hour % 24)

# 클래스 5,6 추출
df_class5 = mul_train_df[mul_train_df['재실인원'] == 5]
df_class6 = mul_train_df[mul_train_df['재실인원'] == 6]

# 클래스 5,6 하나의 DataFrame에 모두 저장
df_combined = pd.concat([df_class5, df_class6], ignore_index=True)

# 재실인원이 0,1,2,3,4인 데이터만 남김
mul_train_df = mul_train_df[mul_train_df['재실인원'].isin([0,1,2,3,4])]

# ADASYN 오버샘플링 적용(0,1,2,3,4만 적용)
adasyn = ADASYN(random_state=28,sampling_strategy='all')
X_oversampled, y_oversampled = adasyn.fit_resample(mul_train_df.drop('재실인원', axis=1), mul_train_df['재실인원'])

# 새로운 오버샘플링된 데이터프레임 생성
df_oversampled = pd.DataFrame(X_oversampled, columns=mul_train_df.drop('재실인원', axis=1).columns)
df_oversampled['재실인원'] = y_oversampled

# 오버샘플링된 데이터프레임을 df로 명명
mul_train_df = df_oversampled

# 재실인원이 1,2,3,4인 데이터만 남기기(0인 데이터들 버리기)
mul_train_df = mul_train_df[mul_train_df['재실인원'].isin([1,2,3,4])]

# 앞서 저장했던 데이터프레임과 결합하여 재실인원이 1,2,3,4,5,6 인 데이터만 남기기
mul_train_df = pd.concat([mul_train_df,df_combined], ignore_index=True)

#최종 선택 컬럼
selected_columns = ['temp', 
                    'co2', 
                    '재실인원']

#최종 학습 데이터
mul_train_data = mul_train_df[selected_columns]

##########################################################################



##########################테스트 데이터 호출(다중 분류)##########################

# CSV 파일을 DataFrame으로 읽어오기 (인코딩 방식 지정)
mul_test_df = pd.read_csv(test_path, encoding='cp949')

# regdate 컬럼을 datetime 형식으로 변환 후 시간을 반올림하여 0~23 사이의 값으로 변환
mul_test_df['regdate'] = pd.to_datetime(mul_test_df['regdate'], format='%Y-%m-%d %H:%M:%S')
mul_test_df['regdate'] = mul_test_df['regdate'].apply(lambda x: (x + datetime.timedelta(minutes=30)).hour % 24)

# 학습 데이터와 동일한 컬럼을 가짐, 최종 테스트 데이터
mul_test_df = mul_test_df[mul_train_data.columns]

# 학습 데이터에서 각 컬럼의 최빈값 계산
mode_values = mul_train_data.mode().iloc[0]

# 테스트 데이터의 결측치를 학습 데이터의 최빈값으로 채우기
mul_test_data = mul_test_df.fillna(mode_values)

##########################################################################



###########################다중 분류 모델 호출 및 최종 결과###########################

# 테스트 데이터셋 
mul_y_test = mul_test_data['재실인원']
mul_X_test = mul_test_data.drop('재실인원', axis=1)

# 저장된 모델 피클 파일을 불러오기
mul_model = joblib.load(mul_model_path)

# 테스트 데이터셋에서 예측값 생성(1,2,3,4)
y_pred_temp = mul_model.predict(mul_X_test)
y_pred_temp = np.ravel(y_pred_temp)

# 최종 병합
# 앞서 0과 1로 이진분류한 결과를 복사
y_fin = np.copy(y_pred)

# 1로 분류된 데이터들만 새롭게 분류된 결과로 대체
y_fin[y_fin == 1] = y_pred_temp[y_fin == 1]

# 예측 결과 평가
# 정확도 계산
accuracy = accuracy_score(mul_y_test, y_fin)

# macro-F1 점수 계산
macro_f1 = f1_score(mul_y_test, y_fin, average='macro')

# weighted-F1 점수 계산
weighted_f1 = f1_score(mul_y_test, y_fin, average='weighted')

# 최종 결과 출력
print("===================================")
print()
print('최종 Accuracy:', accuracy)
print('최종 Macro-F1:', macro_f1)
print('최종 Weighted-F1:', weighted_f1)
print()
print("===================================")

##########################################################################




###############################################################################
####################################다중 분류 끝##################################
###############################################################################



#########################################끝#####################################

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 6.1 MB/s eta 0:00:00
이진분류 Accuracy: 0.9211764705882353
이진분류 ROC AUC: 0.9590969780064986


최종 Accuracy: 0.7921764705882353
최종 Macro-F1: 0.46941579642943543
최종 Weighted-F1: 0.7965403687560086

